In [ ]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from PIL import Image
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.layers import GaussianNoise
from keras.models import Sequential

In [ ]:
mobile = keras.applications.mobilenet.MobileNet()
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)
  
base_model = Sequential()
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=GaussianNoise(0.01, input_shape=(2,))(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='sigmoid')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/google-images-download/images/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/google-images-download/images_val/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=16,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 311 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [ ]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(generator = train_generator,
                  steps_per_epoch=9,validation_data = test_generator,validation_steps = 5,
                  epochs=12)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/12
9/9 [==============================] - 104s 12s/step - loss: 1.0484 - accuracy: 0.5355 - val_loss: 0.3174 - val_accuracy: 0.9000
Epoch 2/12
9/9 [==============================] - 50s 6s/step - loss: 0.1410 - accuracy: 0.9507 - val_loss: 0.6839 - val_accuracy: 0.8375
Epoch 3/12
9/9 [==============================] - 51s 6s/step - loss: 0.1589 - accuracy: 0.9362 - val_loss: 2.4191 - val_accuracy: 0.5875
Epoch 4/12
9/9 [==============================] - 51s 6s/step - loss: 0.0504 - accuracy: 0.9867 - val_loss: 6.8881 - val_accuracy: 0.5250
Epoch 5/12
9/9 [==============================] - 52s 6s/step - loss: 0.0081 - accuracy: 0.9969 - val_loss: 4.9400 - val_accuracy: 0.6125
Epoch 6/12
9/9 [==============================] - 50s 6s/step - loss: 0.0259 - accuracy: 0.9944 - val_loss: 1.1154 - val_accuracy: 0.8875
Epoch 7/12
9/9 [==============================] - 51s 6s/step - loss: 0.0375 - accuracy: 0.9885 - val_loss: 2.2793 - val_accuracy: 0.7500
Epoch 8/12
9/9 [================

In [ ]:
model.save('/content/drive/MyDrive/saved_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/saved_model/assets
